Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [10]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [11]:
pickle_file = '../notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


**Reshape THE IMAGE TO 1-D vector.**

In [12]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [13]:
def build_model(relu_counts, dataset):
    with tf.variable_scope('hidden_layer') as scope:
        try:
            weights = tf.get_variable('weights', [image_size * image_size, relu_counts], initializer = tf.truncated_normal_initializer())
        except ValueError:
            scope.reuse_variables()
            weights = tf.get_variable('weights', [image_size * image_size, relu_counts])
        try:
            biases = tf.get_variable('biases', [relu_counts], initializer = tf.zeros_initializer())
        except ValueError:
            scope.reuse_variables()
            biases = tf.get_variable('biases', [relu_counts])
        hidden_layer_input = tf.matmul(dataset, weights) + biases

    with tf.variable_scope('relu_activation') as scope:
        hidden_layer_output = tf.nn.relu(hidden_layer_input)

    with tf.variable_scope('output_layer') as scope:
        try:
            weights = tf.get_variable('weights', [relu_counts, num_labels], initializer = tf.truncated_normal_initializer())
        except ValueError:
            scope.reuse_variables()
            weights = tf.get_variable('weights', [relu_counts, num_labels])
        try:
            biases = tf.get_variable('biases', [num_labels], initializer = tf.zeros_initializer())
        except ValueError:
            scope.reuse_variables()
            biases = tf.get_variable('biases', [num_labels])
        
        logits = tf.matmul(hidden_layer_output, weights) + biases

    return logits

In [14]:
def train_model(relu_counts, train_dataset, train_labels, learning_rate):
    logits = build_model(relu_counts, train_dataset)
    train_predictions = tf.nn.softmax(logits)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = train_labels))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    return logits, loss, optimizer, train_predictions

In [15]:
def predict(relu_counts, dataset):
    tf_dataset = tf.constant(dataset)
    logits = build_model(relu_counts, tf_dataset)
    predictions = tf.nn.softmax(logits)
    
    return predictions

In [16]:
def accuracy(predictions, labels):
    return (100 * np.sum(np.argmax(predictions, axis = 1) == np.argmax(labels, axis = 1))/predictions.shape[0])

In [19]:
num_steps = 300
BATCH_SIZE = 128
RELU_COUNT = 1024

train_dataset_placeholder = tf.placeholder(tf.float32, shape = [BATCH_SIZE, image_size * image_size])
train_labels_placeholder = tf.placeholder(tf.float32, shape = [BATCH_SIZE, num_labels])
logits, loss, optimizer, predictions = train_model(RELU_COUNT, train_dataset_placeholder, train_labels_placeholder, 0.5)
valid_predict = predict(RELU_COUNT, valid_dataset)
test_predict = predict(RELU_COUNT, test_dataset)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
        batch_data = train_dataset[offset: (offset + BATCH_SIZE), :]
        batch_labels = train_labels[offset: (offset + BATCH_SIZE), :]
        feed_dict = {train_dataset_placeholder: batch_data, train_labels_placeholder: batch_labels}
        
        _, train_loss, _, train_predictions = sess.run([logits, loss, optimizer, predictions], feed_dict = feed_dict)
        if(step % 50 == 0):
            print("Minibatch Loss at step %d: %f" % (step, train_loss))
            print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
    valid_predictions = sess.run(valid_predict)
    test_predictions = sess.run(test_predict)
    print("Valid accuracy: %.1f%%" % accuracy(valid_predictions, valid_labels))
    print("test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Minibatch Loss at step 0: 384.717224
Minibatch accuracy: 10.0%
Minibatch Loss at step 50: 33.732559
Minibatch accuracy: 79.0%
Minibatch Loss at step 100: 16.710146
Minibatch accuracy: 80.0%
Minibatch Loss at step 150: 49.528938
Minibatch accuracy: 75.0%
Minibatch Loss at step 200: 54.372513
Minibatch accuracy: 81.0%
Minibatch Loss at step 250: 21.766233
Minibatch accuracy: 83.0%
Valid accuracy: 79.0%
test accuracy: 86.0%
